참고: https://teddylee777.github.io/langchain/langchain-tutorial-02/

In [1]:
%%shell
pip install langchain
pip install accelerate -U
pip install huggingface_hub transformers datasets
pip install langchain-community langchain-core

## with huggingface inference API
- hugggingface inference endpoint API의 무료 사용량이 제한되어 있어, 실행 안됨 ([pricing](https://huggingface.co/pricing))

In [3]:
# HUGGINGFACEHUB_API_TOKEN= "yout token"

# import os
# os.environ['HUGGINGFACEHUB_API_TOKEN'] = HUGGINGFACEHUB_API_TOKEN



# from langchain import LLMChain
# from langchain.prompts import PromptTemplate
# from langchain.llms import HuggingFaceHub

# # HuggingFace Repository ID
# repo_id = 'mistralai/Mistral-7B-v0.1'

# # 질의내용
# question = "Who is Son Heung Min?"

# # 템플릿
# template = """Question: {question}

# Answer: """

# # 프롬프트 템플릿 생성
# prompt = PromptTemplate(template=template, input_variables=["question"])

# # HuggingFaceHub 객체 생성
# llm = HuggingFaceHub(
#     repo_id=repo_id,
#     model_kwargs={"temperature": 0.2,
#                   "max_length": 128}
# )

# # LLM Chain 객체 생성
# llm_chain = LLMChain(prompt=prompt, llm=llm)

# # 실행
# print(llm_chain.run(question=question))

with huggingface model **local** pipeline

In [5]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate

# HuggingFace Model ID
model_id = "Alphacode-AI/AlphaMist7B-slr-v4-slow2"

# HuggingFacePipeline 객체 생성
llm = HuggingFacePipeline.from_model_id(
    model_id=model_id,
    device=0,               # -1: CPU(default), 0번 부터는 CUDA 디바이스 번호 지정시 GPU 사용하여 추론
    task="text-generation", # 텍스트 생성
    pipeline_kwargs={"temperature": 0.1, "max_length": 8192},
)

# 템플릿
template = """다음은 대한민국 법원에서 내려진 실제 판결 사례들 중 내 상황과 유사한 사례를 가져온 것들이야. 다음 판결 사례들을 기반으로 내가 처한 상황에서 적용될 수 있는 법령 조항들을 모두 알려줘.
다음 두 개의 문단은 판결문들이야.
---
{doc1}
---
{doc2}
---
다음은 내 상황이야.
{query}
이러한 상황에서 단계별로 어떤 법령 조항들을 적용할 수 있을까? 앞서 주어진 판례들을 기반으로 답변해줘.
"""


prompt = PromptTemplate.from_template(template)     # 프롬프트 템플릿 생성
chain = prompt | llm    # 체인 구성



# 문서 가져와서 실행하는거 demo
doc1 = """2023고단1493 판결 [야간주거침입절도, 야간주거침입절도미수, 절도, 절도미수]
## 범죄 사실
피고인은 2023. 3.경 일정한 직업 없이 지내던 중 생활비가 부족해지자 시정되지 않은 길가에 주차된 자동차 또는 빈집에 들어가 물건을 훔치기로 마음먹었다.

피고인은 2023. 3. 초순경 광주 동구 B ○층에 있는 피해자 C의 집 앞에 이르러, 피해자가 장바구니에 담아 현관문 앞에 놓아둔 시가 합계 3만 원 상당의 두부, 토마토, 고춧가루 등 식료품을 들고 가 피해자의 재물을 절취한 것을 비롯하여 그때부터 2023. 3. 하순경까지 사이에 별지 범죄일람표 기재와 같은 방법으로 총 13회에 걸쳐 야간에 타인의 주거에 침입하여 물건을 절취하거나 타인의 자동차 문을 열고 들어가 합계 5,633,000원 상당의 물건을 절취하고, 총 5회에 걸쳐 야간에 타인의 주거에 침입하여 물건을 절취하려 하였으나 범행이 발각되거나, 타인의 자동차 안을 물색하였으나 훔칠 물건을 발견하지 못하여 그 뜻을 이루지 못하는 등 미수에 그쳤다.


## 법령의 적용
각 형법 제329조(절도의 점, 징역형 선택), 각 형법 제330조(야간주거침입절도의 점), 각 형법 제342조, 제329조(절도미수의 점, 징역형 선택), 각 형법 제342조, 제330조(야간주거침입절도미수의 점)
"""

doc2 = """2023고단684 판결 [야간주거침입절도, 야간주거침입절도미수]

## 범죄 사실
1. 야간건조물침입절도

가. 2023. 2. 4.경 범행

피고인은 2023. 2. 4. 23:36 전북 전주시 완산구 B에 있는 피해자 C가 운영하는 'D'에 이르러 시정되지 않은 좌측 셔터문을 열고 들어가 그곳에 보관되어 있던 피해자 소유의 현금 450,000원1) 및 시가 100,000원 상당의 차와 약재를 꺼내어 가지고 갔다.

이로써 피고인은 야간에 타인이 관리하는 건조물에 침입하여 피해자의 재물을 절취하였다.

나. 2023. 2. 10. 23:57경 범행

피고인은 2023. 2. 10. 23:57경 제가항 기재 장소에 이르러 시정되어 있지 않은 좌측 셔터문을 열고 들어가 그곳에 보관되어 있던 피해자 소유의 시가 30,000원 상당의 튀밥, 시가 70,000원 상당의 약초, 시가 10,000원 상당의 보리쌀을 꺼내어 가지고 갔다. 이로써 피고인은 야간에 타인이 관리하는 건조물에 침입하여 피해자의 재물을 절취하였다.

다. 2023. 2. 24. 22:36경 범행

피고인은 2023. 2. 24. 22:36경 제가항 기재 장소에 이르러 시정되어 있지 않은 좌측 셔터문을 열고 들어가 그곳에 보관되어 있던 피해자 소유의 시가 미상의 쌀 1봉지를 꺼내어 가지고 갔다.

이로써 피고인은 야간에 타인이 관리하는 건조물에 침입하여 피해자의 재물을 절취하였다.

2. 야간건조물침입절도미수

피고인은 2023. 3. 2. 22:40경 제가항 기재 장소에 이르러 시정되어 있지 않은 좌측셔터문을 열고 들어가 절취할 물건을 찾던 중 신고를 받고 출동한 경찰관에게 현행범인으로 체포되었다.

이로써 피고인은 야간에 건조물에 침입하여 피해자의 재물을 절취하려다가 그 뜻을 이루지 못하고 미수에 그쳤다.


## 법령의 적용
각 형법 제330조(야간건조물침입절도의 점), 형법 제342조, 제330조(야간건조물침입절도미수의 점)
"""

query = """내 친구가 나한테 캐리어를 맡겼는데, 그 친구가 내가 여행떄문에 집을 비운 동안 그 캐리어가 필요하다고 돌려달라는거야.
난 당연히 지금 여행중이라 캐리어를 건네주기 어렵다고 했지.
근데 친구는 화를 내면서 내 집 도어락 비밀번호를 안다고 하면서, 자기가 우리 집 문을 열고 캐리어를 가져갔어.
그리고 의심하건데 아마 우리 집에 현금이 100만원 정도 있었는데, 이것도 그떄 같이 가져간 것 같거든.
이런 경우에는 내 친구를 어떤 죄목으로 고소할 수 있을까??
"""

print(chain.invoke({"query": query, "doc1": doc1, "doc2": doc2}))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2344 > 2048). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


다음은 대한민국 법원에서 내려진 실제 판결 사례들 중 내 상황과 유사한 사례를 가져온 것들이야. 다음 판결 사례들을 기반으로 내가 처한 상황에서 적용될 수 있는 법령 조항들을 모두 알려줘.
다음 두 개의 문단은 판결문들이야.
---
2023고단1493 판결 [야간주거침입절도, 야간주거침입절도미수, 절도, 절도미수]
## 범죄 사실
피고인은 2023. 3.경 일정한 직업 없이 지내던 중 생활비가 부족해지자 시정되지 않은 길가에 주차된 자동차 또는 빈집에 들어가 물건을 훔치기로 마음먹었다.

피고인은 2023. 3. 초순경 광주 동구 B ○층에 있는 피해자 C의 집 앞에 이르러, 피해자가 장바구니에 담아 현관문 앞에 놓아둔 시가 합계 3만 원 상당의 두부, 토마토, 고춧가루 등 식료품을 들고 가 피해자의 재물을 절취한 것을 비롯하여 그때부터 2023. 3. 하순경까지 사이에 별지 범죄일람표 기재와 같은 방법으로 총 13회에 걸쳐 야간에 타인의 주거에 침입하여 물건을 절취하거나 타인의 자동차 문을 열고 들어가 합계 5,633,000원 상당의 물건을 절취하고, 총 5회에 걸쳐 야간에 타인의 주거에 침입하여 물건을 절취하려 하였으나 범행이 발각되거나, 타인의 자동차 안을 물색하였으나 훔칠 물건을 발견하지 못하여 그 뜻을 이루지 못하는 등 미수에 그쳤다.


## 법령의 적용
각 형법 제329조(절도의 점, 징역형 선택), 각 형법 제330조(야간주거침입절도의 점), 각 형법 제342조, 제329조(절도미수의 점, 징역형 선택), 각 형법 제342조, 제330조(야간주거침입절도미수의 점)

---
2023고단684 판결 [야간주거침입절도, 야간주거침입절도미수]

## 범죄 사실
1. 야간건조물침입절도

가. 2023. 2. 4.경 범행

피고인은 2023. 2. 4. 23:36 전북 전주시 완산구 B에 있는 피해자 C가 운영하는 'D'에 이르러 시정되지 않은 좌측 셔터문을 열고 들어가 그곳에 보관되어 있던 피해자 소유의 현금 450,000원1) 및 시가 100,000원 

test w/o docs

In [7]:
# 템플릿
template = """
다음은 내 상황이야.
{query}
이러한 상황에서 단계별로 어떤 법령 조항들을 적용할 수 있을까?
"""

# 프롬프트 템플릿 생성
prompt = PromptTemplate.from_template(template)

# 체인 구성
chain = prompt | llm



query = """내 친구가 나한테 캐리어를 맡겼는데, 그 친구가 내가 여행떄문에 집을 비운 동안 그 캐리어가 필요하다고 돌려달라는거야.
난 당연히 지금 여행중이라 캐리어를 건네주기 어렵다고 했지.
근데 친구는 화를 내면서 내 집 도어락 비밀번호를 안다고 하면서, 자기가 우리 집 문을 열고 캐리어를 가져갔어.
그리고 의심하건데 아마 우리 집에 현금이 100만원 정도 있었는데, 이것도 그떄 같이 가져간 것 같거든.
이런 경우에는 내 친구를 어떤 죄목으로 고소할 수 있을까??
"""

print(chain.invoke({"query": query}))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



다음은 내 상황이야.
내 친구가 나한테 캐리어를 맡겼는데, 그 친구가 내가 여행떄문에 집을 비운 동안 그 캐리어가 필요하다고 돌려달라는거야. 
난 당연히 지금 여행중이라 캐리어를 건네주기 어렵다고 했지. 
근데 친구는 화를 내면서 내 집 도어락 비밀번호를 안다고 하면서, 자기가 우리 집 문을 열고 캐리어를 가져갔어. 
그리고 의심하건데 아마 우리 집에 현금이 100만원 정도 있었는데, 이것도 그떄 같이 가져간 것 같거든. 
이런 경우에는 내 친구를 어떤 죄목으로 고소할 수 있을까?? 

이러한 상황에서 단계별로 어떤 법령 조항들을 적용할 수 있을까?

1. 먼저 우리 집 문을 열고 캐리어를 가져간 행위는 무단 침입으로 볼 수 있습니다. 이는 형법 제28조에 따라 처벌됩니다. 이 조항은 누군가 다른 사람의 동의 없이 다른 사람의 거주지에 무단으로 들어가는 것을 금지합니다. 이 경우 친구는 우리 집 문을 열고 캐리어를 가져갔으므로 이 조항이 적용될 수 있습니다.
2. 다음으로 우리 집 문을 열고 캐리어를 가져가는 행위는 도난 또는 훔치로 간주될 수 있습니다. 이는 형법 제302조에 따라 처벌됩니다. 이 조항은 누군가 다른 사람의 동의 없이 재산을 훔치는 행위를 규정합니다. 이 경우 친구는 우리 집 문을 열고 캐리어를 가져갔으며 우리 집에 있던 현금도 가져갔으므로 이 조항이 적용될 수 있습니다.
3. 또한 우리 집 문을 열고 캐리어를 가져가는 행위는 사적 침입으로 간주될 수 있습니다. 이는 형법 제293조에 따라 처벌됩니다. 이 조항은 누군가 다른 사람의 동의 없이 다른 사람의 재산에 무단으로 침입하는 행위를 규정합니다. 이 조항은 우리 집 문을 열고 캐리어를 가져가는 행위와 관련이 있을 수 있습니다.
4. 마지막으로 우리 집 문을 열고 캐리어를 가져가는 행위는 사기 또는 속임수로 간주될 수 있습니다. 이는 형법 제301조에 따라 처벌됩니다. 이 조항은 누군가 다른 사람을 속여 그의 재산을 훔치는 행위를 규정합니다. 이 경우 친구는 우리 집 문을 열고 캐리어를 가